In [1]:
import pandas as pd
from IPython.display import Markdown, display

from tulip.data.bloomberg import BloombergClient as bb
from tulip.data.haver import HaverClient as hc
from tulip.data.gs import GSClient as gs
from tulip.data.fred import get_series_from_fred
from tulip.plots import plot_lines, add_plot_theme
from tulip.core import TulipSeries
from tulip.genai import iris

import plotly.offline as pyo

pyo.init_notebook_mode(connected=True)

ctry_iso2 = "JP"

bloomberg_pulls = {}
haver_pulls = {}
weekly_indicators = {}

employment_indicators = {
    "Unemployment Rate (MoM)": "JNUE Index",
}

activity_indicators = {
    "Real Consumption Activity Index": "JNCOREAL Index",
    "Industrial Production": "JNIP Index",
    "Industrial Production (MoM)": "JNIPMOM Index",
    "Real Exports": "JNEIEXPT Index",
    "Real Imports": "JNEIIMPT Index",
    "Housing Starts": "JNHSAN Index",
    "Inventories": "JIPIM&M Index",
}
activity_indicators_collection = []

BLOOMBERG_NOWCAST = "JNBGNCI Index"
activity_indicators_collection = bb.create_collection(activity_indicators)

Haver path setting remains unchanged.




## Japan
### Weekly Indicators
SRI+ Hitotsubashi Consumer Purchase Indexes 
​The SRI-Hitotsubashi Consumer Purchase Index is a high-frequency economic indicator developed by Hitotsubashi University's Institute of Economic Research in collaboration with the National Supermarket Association of Japan and INTAGE Inc. It leverages point-of-sale (POS) data from approximately 6,000 retail outlets across Japan—including supermarkets, drugstores, convenience stores, and discount food stores—to monitor consumer purchasing behavior in near real-time

- POS-CEI (Consumer Expenditure Index): Measures changes in consumer spending.
- POS-UVPI (Unit Value Price Index): Evaluates changes in unit prices, accounting for factors like product size reductions (shrinkflation)


In [2]:
sri_hitotsubashi_consumer_purchase_exp_indexes = {
    "POS-CEI Sales YoY": "P158ETSY@INTWKLY",
    "POS-CEI Price YoY": "P158ETPP@INTWKLY",
    "Unit Value Price YoY": "P158UTUY@INTWKLY",
}

sri_hitotsubashi_consumer_purchase_exp_indexes_collection = hc.create_collection(
    sri_hitotsubashi_consumer_purchase_exp_indexes
)

sri_hitotsubashi_consumer_purchase_exp_indexes_collection.dashboard.table()


,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
POS-CEI Sales YoY,3.4,2025-10-27,1.2,2.2,0.54,1.1,2.5,2025-11-12 19:22:00
POS-CEI Price YoY,2.4,2025-10-27,2.3,0.16,1.1,-0.47,0.73,2025-11-12 19:22:00
Unit Value Price YoY,3.5,2025-10-27,3.4,0.02,0.05,0.32,0.94,2025-11-12 19:22:00


In [3]:
sri_hitotsubashi_consumer_purchase_exp_indexes_collection.dashboard.plots(
    years_limit=5, mma=6, show_0=True
)

### GS Current Activity Indicator (GS Nowcast)

In [4]:
cai_series_soft_vs_hard = gs.get_CAI_series(
    geographyId=ctry_iso2,
    metricName=[
        "CAI_HEADLINE",
        "CAI_CONTRIBUTION_TYPE_HARD",
        "CAI_CONTRIBUTION_TYPE_SOFT",
    ],
    startDate="1980-01-01",
)
cai_series_soft_vs_hard = cai_series_soft_vs_hard.set_index("metricName", append=True)[
    "metricValue"
].unstack("metricName")
cai_series_soft_vs_hard.columns = ["Hard", "Soft", "Headline"]
cai_plot = plot_lines(
    cai_series_soft_vs_hard,
    show_0=True,
    title=f"<b>{ctry_iso2} Current Activity Indicator</b> Updated: {pd.Timestamp.today().strftime('%Y-%m-%d')}",
    years_limit=4,
)
cai_plot

### Activity Indicators

In [5]:
activity_indicators_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Quote Units,Updated
Real Consumption Activity Index,99.8,2025-09-30,99.3,0.48,0.27,-1.3,0.38,-,2025-11-12 19:22:00
Industrial Production,102.8,2025-09-30,100.6,2.2,1.0,0.60,1.6,-,2025-11-12 19:22:00
Industrial Production (MoM),2.2,2025-09-30,-1.5,3.7,1.3,-0.10,1.5,-,2025-11-12 19:22:00
Real Exports,119.1,2025-09-30,115.8,3.3,1.4,-3.4,1.00,-,2025-11-12 19:22:00
Real Imports,112.1,2025-09-30,109.0,3.1,1.2,3.0,2.4,-,2025-11-12 19:22:00
Housing Starts,0.73,2025-09-30,0.71,0.02,0.26,-0.08,-0.07,-,2025-11-12 19:22:00
Inventories,99.6,2025-09-30,99.1,0.50,0.47,-1.3,-2.7,-,2025-11-12 19:22:00


In [6]:
activity_indicators_collection.dashboard.plots(years_limit=5, mma=6)

### Employment

In [7]:
japan_labor_data = {
    "Unemployment rate": "FLUEDR@JAPAN",
    "Labour force participation rate": "sa(FLPR@JAPAN)",
    "New Jobs to Applicants": "JANOR2@JAPAN",
}

employment = hc.create_collection(japan_labor_data)
employment[0].good_is = -1
iris_employment = iris.summarize(employment, "What trends are you seeing in japan? ")
iris_employment.html()

In [8]:
employment.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
Unemployment rate,2.6,2025-09-30,2.6,0.00,0.00,0.20,0.10,2025-11-12 19:23:00
Labour force participation rate,63.9,2025-09-30,63.8,0.08,0.42,0.22,0.69,2025-11-12 19:23:00
New Jobs to Applicants,2.1,2025-09-30,2.1,-0.01,-0.17,-0.16,-0.06,2025-11-12 19:23:00


In [9]:
employment.dashboard.plots(tick_suffix="%")

### Prices

In [10]:
japan_cpi_data = {
    "CPI Headline": "JYCIJ@JAPAN",  # This is the one bloomberg takes Also called nationwide
    "CPI Core": "JYCIJE@JAPAN",
    "CPI chained index, NSA": "JPCIJY@JAPAN",
    "CPI: Trimmed mean": "JPNPCT@JAPAN",
    "CPI: Mode": "JPNPCM@JAPAN",
    "CPI: Weighted median": "JPNPCW@JAPAN",
    "Diffusion index": "JPNPCD@JAPAN",
    "Share of increasing items": "JPNPCDA@JAPAN",
    "Share of decreasing items": "JPNPCDB@JAPAN",
}

japan_inflation_collection = hc.create_collection(japan_cpi_data)
japan_inflation_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
CPI Headline,2.9,2025-09-30,2.7,0.20,0.34,-0.80,0.40,2025-11-12 19:23:00
CPI Core,2.9,2025-09-30,2.7,0.20,0.43,-0.10,0.50,2025-11-12 19:23:00
"CPI chained index, NSA",3.0,2025-09-30,2.7,0.30,0.80,-0.60,0.60,2025-11-12 19:23:00
CPI: Trimmed mean,2.1,2025-09-30,2.0,0.06,0.49,-0.08,0.39,2025-11-12 19:23:00
CPI: Mode,1.7,2025-09-30,1.9,-0.20,-1.8,0.50,0.30,2025-11-12 19:23:00
CPI: Weighted median,1.4,2025-09-30,1.1,0.30,2.7,-0.04,0.52,2025-11-12 19:23:00
Diffusion index,60.0,2025-09-30,65.5,-5.6,-1.5,-3.1,2.1,2025-11-12 19:23:00
Share of increasing items,77.6,2025-09-30,80.1,-2.5,-1.3,-1.1,1.5,2025-11-12 19:23:00
Share of decreasing items,17.6,2025-09-30,14.6,3.1,1.5,1.9,-0.57,2025-11-12 19:23:00


In [11]:
cpi_metrics_df = japan_inflation_collection.df.rename(
    columns={v.split("@")[0].lower(): k for k, v in japan_cpi_data.items()}
).drop(
    columns=[
        "Diffusion index",
        "Share of increasing items",
        "Share of decreasing items",
    ],
    errors="ignore",
)

In [12]:
fig = plot_lines(
    cpi_metrics_df,
    years_limit=5,
    show_0=True,
    tick_suffix="%",
    logo=False,
    title="<b>Japan CPI Metrics</b>",
)

fig.update_layout(
    title_y=0.97,
)

fig.update_traces(selector={"name": "CPI Core"}, line=dict(width=4))
fig.update_traces(selector={"name": "CPI: Mode"}, visible="legendonly")
fig.update_traces(selector={"name": "CPI: Trimmed mean"}, visible="legendonly")
fig.update_traces(selector={"name": "CPI: Weighted median"}, visible="legendonly")


`````{admonition} Which CPI measure is the most relevant for the BOJ's mandate?
:class: note

The actual mandate as stated on their website is **headline CPI** - 2%. But because fresh food prices are more volatile (and Jin from Observatory group confirmed that), they actually look at the **core CPI** (headline less fresh food). That's the one they reference in their outlook report.  Adn that seems to be the effective one that they care about  (https://www.boj.or.jp/en/mopo/outlook/gor2507b.pdf) 
 
However, as a way to gauge what's going on and to keep track of their progress, etc., they have mentioned core-core CPI and the trimmed mean/weighted median to get a sense of the underlying pressures. For example, they have referenced the trimmed mean being at 2.5% and mode and wieghted median being around 1.5% as a way to assess whether price increases aer broadly based or limited to a few volatile categories. 
`````


Ad-hoc chart we have used in certain marketing materials.

In [13]:
forward = bb.bdh("G0018 1Y1Y BLC2 Curncy")
cpi = bb.bdh("JNCPIYOY Index")

plot_lines(
    [
        forward.iloc[:, 0].rename("1Y1Y forward rate"),
        cpi.iloc[:, 0].rename("Japan Nationwide CPI"),
    ],
    years_limit=4,
    tick_suffix="%",
    title="Japanese 1yr 1yr forward yield vs inflation",
    source="Kate Capital",
)

In [14]:
Markdown(f"_Notebook updated at {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}_")

_Notebook updated at 2025-11-12 19:23_